In [42]:
import subprocess as sub
import sys
import ads
import pandas as pd

In [43]:
# To create a database with features and their topological structure
# One needs to create a csv file with 
# https://planetarynames.wr.usgs.gov/nomenclature/AdvancedSearch
# and pass it to the database_creation script
# IMPORTANT: feature ID and feature name should be present in csv file

In [44]:
script_name = "database_merged.py"
csv_file_name = "Mars_short.csv"

try:
    ret = sub.call(["python", script_name, csv_file_name], shell = True)
    if (ret < 0):
        print("Child process was terminated by signal", -ret, file=sys.stderr)
    else:
        print("Child returned", ret, file=sys.stderr)
except OSError as e:
    print("Execution failed: ", e, file=sys.stderr)

Child returned 0


In [45]:
# now database is built and we want to provide speific queries to ADS
authors = ["Comer,R.P.", "Solomon,S.C.", "Head,J.W."]
toponym = ["Mons Olympus"]
years = range(1978, 2003)
query_list = ["", "", ""]

if (toponym):
    for idx in range(len(toponym) - 1):
        query_list[0] += toponym[idx] + " OR "
    query_list[0] += toponym[-1]

if (authors):
    for idx in range(len(authors) - 1):
        query_list[1] += "author:" + authors[idx] + " OR "
    query_list[1] += "author:" + authors[-1]
        
if (years):
    query_list[2] = "year:[" + str(years.start) + " TO " + str(years.stop) + "]" 

query = ' AND '.join(filter(None, query_list))        

In [46]:
print(query)
example_results = []
try:
    example_results = list(ads.SearchQuery(q=query, fl=['title', 'author', 'year', 'pub', 'bibcode']))
except (ads.exceptions.APIResponseError, ads.exceptions.SolrResponseParseError) as e:
    "Error: {}".format(e)

Mons Olympus AND author:Comer,R.P. OR author:Solomon,S.C. OR author:Head,J.W. AND year:[1978 TO 2003]


In [48]:
# representing the results in a table
pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame(example_results)

In [49]:
df

,0
0,"<Unknown author et al. 1985, 1985RvGeo..23...61C>"
1,"<Unknown author et al. 1981, 1981LPI....12..166C>"
2,"<Unknown author et al. 1981, 1981LPICo.441..244S>"
3,"<Unknown author et al. 1980, 1980LPI....11..171C>"
